### 1.1. Explode CORD-NER-corpus.json file to dataframe (.csv)
- Since the dataset is too big, this is done in segments.

In [ ]:
import pandas as pd
import json
import time

start_time = time.time()

In [ ]:
with open('CORD-NER-corpus.json', 'r') as myfile:
    corpus_data = myfile.read()

In [ ]:
corpus_list = corpus_data.split('\n')

In [ ]:
df = pd.DataFrame()
df['doc_id'] = ''
df['sent_id'] = ''
df['sent_tokens'] = ''
df['word_id'] = ''
df['tag'] = ''

In [ ]:
# last element, corpus_list[29500] = '', which is empty. So, go up to len -1
for idx in range(29300, len(corpus_list)-1):
#for idx in range(29300, 27000):
    data = json.loads(corpus_list[idx])
    df_1 = pd.DataFrame(data)

    # no sentence in doc_id = 18434
    if df_1.empty:
        continue

    df_2 = pd.json_normalize(df_1['sents'])
    df_2['word_id'] = df_2['sent_tokens'].apply(lambda x: range(len(x)))
    df_explode = df_2.explode(['sent_tokens', 'word_id'])
    df_explode['doc_id'] = idx

    frames = [df, df_explode]
    df = pd.concat(frames)

    if ((idx % 100) == 0):
        print('idx: ', idx, ' & continuing ... ')

idx:  29300  & continuing ... 
idx:  29400  & continuing ... 


In [ ]:
df['tag'] = 'O'

In [ ]:
df

,doc_id,sent_id,sent_tokens,word_id,tag
0,29300,0,part,0,O
0,29300,0,of,1,O
0,29300,0,containing,2,O
0,29300,0,infectious_disease,3,O
0,29300,0,outbreaks,4,O
...,...,...,...,...,...
66,29499,66,in,29,O
66,29499,66,interpreting,30,O
66,29499,66,this,31,O
66,29499,66,result,32,O


In [ ]:
end_time = time.time()

estimate_runtime = round((end_time-start_time)/60)
print('Estimate runtime: ', (end_time-start_time)/60, ' minutes.')

Estimate runtime:  0.9166793107986451  minutes.


In [ ]:
file_name = 'corpus_without_tag_29300-end_' + str(estimate_runtime) + 'min.csv'
df.to_csv(file_name)